# Processing Raw Sequences

Scripts used to trim raw sequences, check quality, map to ref.

## 1. trimming 

using `trim-galore` - [documentation](https://github.com/FelixKrueger/TrimGalore)

This code uses an array to run jobs in parallel

In [ ]:
#!/bin/bash
#SBATCH --job-name=trim_galore_array
#SBATCH -c 4
#SBATCH --mem=16G
#SBATCH -p cpu
#SBATCH -t 12:00:00
#SBATCH --array=1-120
#SBATCH -o slurm-%A_%a.out
#SBATCH --mail-type=END,FAIL

#-----------------modules-----------------#
module load conda/latest

conda activate cutadapt
# trim-galore is already installed in this env 

#---------------change wd----------------#

# to scratch workspace with downloaded seqs

cd /scratch4/workspace/julia_mcdonough_student_uml_edu-novogene_dwnld

#-----------------commands----------------#

# parent directory containing sample subdirectories
PARENT_DIR="/scratch4/workspace/julia_mcdonough_student_uml_edu-novogene_dwnld/01.RawData"

# output dir for all trimmed files
OUTDIR="/scratch4/workspace/julia_mcdonough_student_uml_edu-novogene_dwnld/trimmed_all"

SAMPLE=$(sed -n "${SLURM_ARRAY_TASK_ID}p" /scratch4/workspace/julia_mcdonough_student_uml_edu-novogene_dwnld/sample_dirs.txt)

R1=("$SAMPLE"/*_1*.fq.gz)
R2=("$SAMPLE"/*_2*.fq.gz)

echo "Running Trim Galore on: $SAMPLE"

trim_galore --paired --fastqc -j 4 -o "$OUTDIR" "${R1[@]}" "${R2[@]}"


## 2. check quality
using FastQC and MultiQC to check quality after trimming adapters

**2a. FastQC** to generate quality assessment files

In [ ]:
#!/bin/bash
#SBATCH --job-name=fastqc_array
#SBATCH -c 4                 # cores per task
#SBATCH --mem=16G             # memory per node
#SBATCH -p cpu
#SBATCH -t 12:00:00
#SBATCH --array=1-120         # number of array tasks
#SBATCH -o slurm-%A_%a.out
#SBATCH --mail-type=END,FAIL

# Load conda and activate environment
module load conda/latest
conda activate fastqc

# Set working directories
INPUT_DIR="/scratch4/workspace/julia_mcdonough_student_uml_edu-novogene_dwnld/trimmed_all"
OUTPUT_DIR="/scratch4/workspace/julia_mcdonough_student_uml_edu-novogene_dwnld/fastqc"
cd "$INPUT_DIR"

# Number of files per task
FILES_PER_TASK=4

# Compute which lines (files) this array task will process
START=$(( (SLURM_ARRAY_TASK_ID - 1) * FILES_PER_TASK + 1 ))
END=$(( SLURM_ARRAY_TASK_ID * FILES_PER_TASK ))

# Loop over assigned files
for i in $(seq $START $END); do
    FILE=$(sed -n "${i}p" fq_files.txt)
    if [ -n "$FILE" ]; then  # skip if line is empty
        echo "Processing $FILE"
        fastqc -t 2 -o "$OUTPUT_DIR" "$FILE"
    fi
done

**2b. MultiQC** to view all 120 samples (480 files) at once

>multiqc runs quickly, so can be done in terminal and don't need to submit a job. both the html and zip files need to be in the same directory.

In [ ]:
multiqc .

## 3. alignment

using `hisat2` ([manual](https://daehwankimlab.github.io/hisat2/manual/)) - following pipeline from [how to page](https://daehwankimlab.github.io/hisat2/howto/)

(using hisat2 over bowtie2 bc bowtie2 isn't splice aware)

#### 3a. build genome index with exons and splice sites
download reference genome (genome.fa) and GTF file (to make exon, splice site file; genome.gtf)

In [ ]:
wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/002/022/765/GCF_002022765.2_C_virginica-3.0/GCF_002022765.2_C_virginica-3.0_genomic.gff.gz
gunzip GCF_002022765.2_C_virginica-3.0_genomic.gff.gz

make exon and splice sites files from GTF file

In [ ]:
# Extract splice sites
hisat2_extract_splice_sites.py GCF_002022765.2_C_virginica-3.0_genomic.gff > oyster.ss

# Extract exons
hisat2_extract_exons.py GCF_002022765.2_C_virginica-3.0_genomic.gff > oyster.exon


Build HFM index - with exon and splice site info using the files above

(HFM = hierarchical FM index for a reference genome)

In [ ]:
#!/bin/bash
#SBATCH --job-name=hisat2_build
#SBATCH -c 8                 # cores per task
#SBATCH --mem=64G             # memory per node
#SBATCH -p cpu
#SBATCH -t 1:00:00
#SBATCH --cpus-per-task=16     
#SBATCH -o hisat2_build.log
#SBATCH --mail-type=END,FAIL

#-----------------modules-----------------#
module load conda/latest

conda activate hisat2-env

#---------------change wd----------------#

cd /work/pi_sarah_gignouxwolfsohn_uml_edu/julia_mcdonough_student_uml_edu/ref_files

#-----------------commands----------------#

hisat2-build -p 16 \
--exon oyster.exon \
--ss oyster.ss \
GCF_002022765.2_C_virginica-3.0_genomic.fna oyster_index


#### 3b. align sequences to genome index

In [ ]:
#!/bin/bash
#SBATCH --job-name=hisat2_align
#SBATCH --cpus-per-task=16
#SBATCH --mem=32G
#SBATCH -p cpu
#SBATCH -t 32:00:00
#SBATCH -o hisat2_align_%j.log
#SBATCH --mail-type=END,FAIL

#-----------------modules-----------------#

module load conda/latest

conda activate hisat2-env

#-----------------set paths----------------#

INDEX="/work/pi_sarah_gignouxwolfsohn_uml_edu/julia_mcdonough_student_uml_edu/ref_files/hisat2_index"
INPUT_DIR="/scratch4/workspace/julia_mcdonough_student_uml_edu-novogene_dwnld/trimmed_all"
OUTPUT_DIR="/scratch4/workspace/julia_mcdonough_student_uml_edu-novogene_dwnld/hisat2-align"

#-----------------commands----------------#

for R1 in ${INPUT_DIR}/*_gi_1_val_1.fq.gz; do
    # Remove _gi_1_val_1.fq.gz to get sample base name
    SAMPLE=$(basename $R1 _gi_1_val_1.fq.gz)
    
    # Construct R2 with gi_2
    R2="${INPUT_DIR}/${SAMPLE}_gi_2_val_2.fq.gz"
    
    # Check if R2 exists
    if [[ ! -f "$R2" ]]; then
        echo "Warning: R2 file not found: $R2"
        continue
    fi
    
    echo "Aligning $SAMPLE..."
    echo "R1: $R1"
    echo "R2: $R2"
    
    hisat2 -p 16 \
        -x $INDEX/oyster_index \
        -1 $R1 \
        -2 $R2 \
        -S ${OUTPUT_DIR}/${SAMPLE}.sam \
        2> ${OUTPUT_DIR}/${SAMPLE}.log
done

the above code takes ~32 hours to finish - below is an (untested!) code for running multiple alignments in parallel with an array job

In [ ]:
#!/bin/bash
#SBATCH --job-name=hisat2_align
#SBATCH --array=1-120%10
#SBATCH --cpus-per-task=16
#SBATCH --mem=32G
#SBATCH -p cpu
#SBATCH -t 6:00:00
#SBATCH -o logs/hisat2_align_%A_%a.log
#SBATCH --mail-type=END,FAIL

#-----------------modules-----------------#
module load conda/latest
conda activate hisat2-env

#-----------------set paths----------------#
INDEX="/work/pi_sarah_gignouxwolfsohn_uml_edu/julia_mcdonough_student_uml_edu/ref_files/hisat2_index"
INPUT_DIR="/scratch4/workspace/julia_mcdonough_student_uml_edu-novogene_dwnld/trimmed_all"
OUTPUT_DIR="/scratch4/workspace/julia_mcdonough_student_uml_edu-novogene_dwnld/hisat2-align"

# Get list of R1 files
R1_FILES=("${INPUT_DIR}"/*_gi_1_val_1.fq.gz)

# Get the R1 file for this array task
R1="${R1_FILES[$SLURM_ARRAY_TASK_ID-1]}"

# Get sample base name
SAMPLE=$(basename "$R1" _gi_1_val_1.fq.gz)

# Construct R2 filename
R2="${INPUT_DIR}/${SAMPLE}_gi_2_val_2.fq.gz"

# Check if R2 exists
if [[ ! -f "$R2" ]]; then
    echo "Error: R2 file not found: $R2"
    exit 1
fi

#-----------------commands----------------#
echo "Aligning $SAMPLE..."
echo "R1: $R1"
echo "R2: $R2"

hisat2 -p 16 \
    -x "$INDEX/oyster_index" \
    -1 "$R1" \
    -2 "$R2" \
    -S "${OUTPUT_DIR}/${SAMPLE}.sam" \
    2> "${OUTPUT_DIR}/${SAMPLE}.log"

## 4. convert SAM to BAM
using `samtools` ([documentation](https://www.htslib.org/doc/samtools-view.html)) to convert SAM files to BAM and sort by genomic coordinates 

In [ ]:
#!/bin/bash
#SBATCH --job-name=sam2bam
#SBATCH --array=1-120
#SBATCH --cpus-per-task=4
#SBATCH --mem=8G
#SBATCH -p cpu
#SBATCH -t 4:00:00
#SBATCH -o logs/sam2bam_%A_%a.log
#SBATCH --mail-type=END,FAIL
#-----------------modules-----------------#

module load samtools/1.19.2

#-----------------set paths----------------#

INPUT_DIR="/scratch4/workspace/julia_mcdonough_student_uml_edu-novogene_dwnld/hisat2-align/"
OUTPUT_DIR="/scratch4/workspace/julia_mcdonough_student_uml_edu-novogene_dwnld/bam_files/"

# Get list of SAM files
SAMPLES=("$INPUT_DIR"*.sam)
SAM_FILE="${SAMPLES[$SLURM_ARRAY_TASK_ID-1]}"

#-----------------commands----------------#

# Convert and sort (by genomic coordinate)
BASE=$(basename "$SAM_FILE" .sam)
samtools view -@ 4 -bS "$SAM_FILE" | samtools sort -@ 4 -o "${OUTPUT_DIR}${BASE}.sorted.bam"
samtools index "${OUTPUT_DIR}${BASE}.sorted.bam"

## 5. read counting
with `Subread-featureCounts' ([manual](https://subread.sourceforge.net/featureCounts.html)) to generate counts matrix


featureCounts recommends to use the same GTF/GFF file that was used for alignment - so using GFF file, but removing the first couple of lines that start with "#" so that featureCounts can read it correctly

In [ ]:
# remove #s in first couple of lines of gff file
grep -v '^#' GCF_002022765.2_C_virginica-3.0_genomic.gff > cv.gff

In [ ]:
#!/bin/bash
#SBATCH --job-name=featureCounts
#SBATCH --array=1-120          
#SBATCH --cpus-per-task=8      # more threads for featureCounts
#SBATCH --mem=16G               
#SBATCH -p cpu
#SBATCH -t 1:00:00              
#SBATCH --output=logs/featureCounts_%A_%a.log
#SBATCH --mail-type=END,FAIL
#-----------------modules-----------------#

module load conda/latest
conda activate subread_env

#-----------------set paths----------------#

INPUT_DIR="/scratch4/workspace/julia_mcdonough_student_uml_edu-novogene_dwnld/bam_files/"
OUTPUT_DIR="/work/pi_sarah_gignouxwolfsohn_uml_edu/julia_mcdonough_student_uml_edu/ce24_rnaseq"

BAM_LIST="$INPUT_DIR/bam_list.txt"

GFF_FILE="/work/pi_sarah_gignouxwolfsohn_uml_edu/julia_mcdonough_student_uml_edu/ref_files/genome/cv.gff"

#-----------------commands----------------#

# Get the BAM file for this array task
BAM_FILE="$INPUT_DIR/$(sed -n "${SLURM_ARRAY_TASK_ID}p" "$BAM_LIST")"

SAMPLE_NAME=$(basename "$BAM_FILE" .sorted.bam)

# Run featureCounts
featureCounts \
-T 8 \
-a "$GFF_FILE" \
-g ID \
-t gene \
-p \
-B \
-C \
-s 0 \
-o "$OUTPUT_DIR/counts_${SAMPLE_NAME}.txt" \
"$BAM_FILE"

annotation of featureCounts options:
```
featureCounts \
  -T 8 \                  # threads
  -a "$gff.file" \        # GTF annotation
  -g ID \                 # gene identifier attribute
  -t gene \               # feature type
  -p \                    # paired-end
  -B \                    # require both ends mapped
  -C \                    # ignore chimeric fragments
  -s 0 \                  # unstranded
  -o "$OUTPUT_DIR/counts_${SLURM_ARRAY_TASK_ID}.txt" \
  "$BAM_FILE"
```

the output of this will be multiple .txt files, two for each sample (one containing the counts matrix, the other containing summaries about read counting)

In [ ]:
#!/bin/bash

INPUT_DIR="/work/pi_sarah_gignouxwolfsohn_uml_edu/julia_mcdonough_student_uml_edu/ce24_rnaseq/featureCounts/"
OUTPUT_FILE="//project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_2024/CE24_RNA-seq/processing/qc_outputs/featureCounts_summary.csv"

# Get header from the first file
first_file=$(ls $INPUT_DIR/*.summary | head -n 1)
awk 'NR>1 {print $1}' "$first_file" | paste -sd ',' - > "$OUTPUT_FILE.header"

# Start the CSV with "Sample" column
echo "Sample,$(cat $OUTPUT_FILE.header)" > "$OUTPUT_FILE"

# Loop over all files
for f in $INPUT_DIR/*.summary; do
    sample=$(basename "$f" .txt.summary)  # extract sample name
    awk 'NR>1 {print $2}' "$f" | paste -sd ',' - | awk -v s="$sample" '{print s "," $0}' >> "$OUTPUT_FILE"
done

# Clean up
rm "$OUTPUT_FILE.header"
